In [9]:
import sys
sys.path.append(r'C:\Users\Mark\Desktop\BXA\HealthcareAnalyticTools')

In [1]:
#%load load.py

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

from pyexcel_ods import get_data as OpenOffice

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

## Read_FDB

In [1]:
def Read_FDB(path=r"C:\Users\Mark\Desktop\BXA\HealthcareAnalyticTools\src\FDB_Base.xlsx"):

    dfFDB = pd.read_excel(path, sheet_name='FirstDatabankMedicationBASE')
    print("NumRecs: \n", dfFDB.shape)
    print("Is the NDC unique: \n" + str(dfFDB['NDC'].is_unique))
    
    ## Dtype for NDC
    dfFDB['NDC'] = dfFDB['NDC'].astype(str).str.strip().str.zfill(11)
    
    ## Rename Drug Columns
    dfFDB.rename(columns={'BrandNM':'DrugNameFDB', 'GenericNM':'GenericNameFDB', 'AverageWholesalePriceAMT':'AvgWholesalePriceFDBAmt'}, inplace=True)

    ## Select columns for indication, if it needs an underscore, add one

    cols_dfFDB = pd.Series(dfFDB.columns)

    m = cols_dfFDB.str.contains('Indication', case=False)

    indication_columns_orig = cols_dfFDB[m]
    indication_columns_new = cols_dfFDB[m].str.replace(" ", "_")

    rename = dict(zip(indication_columns_orig, indication_columns_new))
    dfFDB.rename(columns=rename, inplace=True)
    
    ## Join the Indications into one Column
    def join_indication_columns(r):
        r = r.tolist()
        l = [i for i in r if i != '']
        j = ' | '.join(l)
        return j

    dfFDB['IndicationsFDB'] = dfFDB[indication_columns_new]\
                                .fillna('')\
                                .apply(lambda c: c.str.strip())\
                                .apply(join_indication_columns, axis=1)
    
    print("Key dfFDB column names:\n", ['NDC', 'DrugNameFDB', 'GenericNameFDB', 'IndicationsFDB', 'AvgWholesalePriceFDBAmt'])
    
    return dfFDB

In [ ]:
dfFDB = Read_FDB()

## Read JCodeRxLookUp

In [3]:
def Read_JCodeRxLookUp(path=r"C:\Users\Mark\Desktop\BXA\HealthcareAnalyticTools\src\JCode List.xlsx", Return='NoDups'):
    """ Returns The JCodeRxLookUp table  
    -------------------------------------  
    Return: Some JCodes are duplicated, do you want 'Dups' or 'NoDups'? Default is 'NoDups'.  
    -------------------------------------  
    """

    df = pd.read_excel(path)
    print("NumRecs: \n", df.shape)
    print("Is the JCode unique: \n" + str(df['JCode'].is_unique))
    
    ## Dtypes
    df['JCode'] = df['JCode'].str.strip()
    
    ## Rename Columns
    df.rename(columns={'JCode_Description':'JCodeRxName'}, inplace=True)
    
    ## Select columns for indication, if it needs an underscore, add one

    cols_df = pd.Series(df.columns)

    m = cols_df.str.contains('Indication', case=False)

    indication_columns_orig = cols_df[m]
    indication_columns_new = cols_df[m].str.replace(" ", "_")

    rename = dict(zip(indication_columns_orig, indication_columns_new))
    df.rename(columns=rename, inplace=True)
    
    ## Join the Indications into one Column
    def join_indication_columns(r):
        r = r.tolist()
        l = [i for i in r if i != '']
        j = ' | '.join(l)
        return j

    df['IndicationsJCode'] = df[indication_columns_new]\
                                .fillna('')\
                                .apply(lambda c: c.str.strip())\
                                .apply(join_indication_columns, axis=1)
    
    print("Key dfJCodeLookup column names:\n", ['JCode', 'IndicationsJCode'])
    
    Dups = dfJCodeLookup[dfJCodeLookup.duplicated('JCode', keep=False)].sort_values('JCode').copy()
    NoDups = dfJCodeLookup[~dfJCodeLookup.index.isin(dups.index)].copy()
    
    if Return == 'Dups':
        return Dups
    elif Return == 'NoDups':
        return NoDups